# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [83]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [84]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,georgetown,5.4112,100.3354,80.53,73,20,2.30,MY,1741044533
1,1,waitangi,-43.9535,-176.5597,64.81,97,100,5.01,NZ,1741044537
2,2,isafjordur,66.0755,-23.1240,27.61,93,100,18.10,IS,1741044541
3,3,aksay,47.2600,39.8700,26.82,86,54,6.55,RU,1741044545
4,4,grytviken,-54.2811,-36.5092,46.38,97,100,10.29,GS,1741044549


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [85]:
#%%capture --no-display

# Configure the map plot
humidityPlot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504, 
    frame_width = 896,
    hover_cols = ["City", "Humidity"]
)

# Display the map
humidityPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [86]:
# Narrow down cities that fit criteria and drop any results with null values - 
# max temp: 27C ~ 80.6 F
# min temp: 21 C ~ 69.8 F
# wind < 4.5
# cloudiness = 0

idealCityDF = city_data_df.loc[
    (city_data_df["Max Temp"] < 80.6) & (city_data_df["Max Temp"] > 69.8)
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
idealCityDF = idealCityDF.dropna()

# Display sample data
print(len(idealCityDF))
idealCityDF.head()

2


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
440,440,cam ranh,11.9214,109.1591,75.16,94,0,0.00,VN,1741046457
442,442,vicuna,-30.0319,-70.7081,71.96,49,0,2.93,CL,1741046466


### Step 3: Create a new DataFrame called `hotel_df`.

In [87]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = idealCityDF[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
440,cam ranh,VN,11.9214,109.1591,94,
442,vicuna,CL,-30.0319,-70.7081,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [88]:
# Set parameters to search for a hotel
radius = 10000
params = {
     "categories": "accommodation.hotel",
     "apiKey": geoapify_key,
     "limit" : 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"] 
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df


Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Khách sạn Thái Bình', 'country': 'Vietnam', 'country_code': 'vn', 'state': 'Khánh Hòa Province', 'city': 'Cam Ranh', 'suburb': 'Phường Cam Thuận', 'street': 'Hùng Vương', 'housenumber': '2013', 'lon': 109.1542493, 'lat': 11.9181403, 'formatted': 'Thai Binh Cam Ranh Hotel, 2013, Hùng Vương, Phường Cam Thuận, Cam Ranh, Khánh Hòa Province, Vietnam', 'address_line1': 'Thai Binh Cam Ranh Hotel', 'address_line2': '2013, Hùng Vương, Phường Cam Thuận, Cam Ranh, Khánh Hòa Province, Vietnam', 'categories': ['accommodation', 'accommodation.hotel', 'internet_access'], 'details': ['details', 'details.facilities'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Khách sạn Thái Bình', 'osm_id': 5417777521, 'name:en': 'Thai Binh Cam Ranh Hotel', 'to

,City,Country,Lat,Lng,Humidity,Hotel Name
440,cam ranh,VN,11.9214,109.1591,94,Khách sạn Thái Bình


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [89]:
# %%capture --no-display
# Configure the map plot
hotelPlot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.7,
    tiles = "OSM",
    frame_height = 504, 
    frame_width = 896,
    hover_cols = ["City", "Humidity","Hotel Name"]
)

# Display the map
hotelPlot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)